In [19]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [20]:
from keras.datasets import cifar10

In [21]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [22]:

print ("Training data:")
print ("Number of examples: ", X_train.shape[0])
print ("Number of channels:",X_train.shape[3]) 
print ("Image size:", X_train.shape[1], X_train.shape[2])
print
print ("Test data:")
print ("Number of examples:", X_test.shape[0])
print ("Number of channels:", X_test.shape[3])
print ("Image size:", X_test.shape[1], X_test.shape[2])

Training data:
Number of examples:  50000
Number of channels: 3
Image size: 32 32
Test data:
Number of examples: 10000
Number of channels: 3
Image size: 32 32


In [23]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 33, 33, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32) 

In [31]:
base_model=MobileNet(input_shape=(32,32,3), weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)

In [32]:
model=Model(inputs=base_model.input,outputs=preds)

In [33]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 33, 33, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32)       

In [34]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [35]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
one_hot_labels = keras.utils.to_categorical(y_train, num_classes=10)

In [36]:
model.fit(X_train, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 215s 138ms/step - loss: 1.5856 - accuracy: 0.4524
Epoch 2/10
1563/1563 [==============================] - 219s 140ms/step - loss: 1.3177 - accuracy: 0.5523
Epoch 3/10
1563/1563 [==============================] - 215s 138ms/step - loss: 1.2060 - accuracy: 0.5946
Epoch 4/10
1563/1563 [==============================] - 215s 137ms/step - loss: 1.1189 - accuracy: 0.6224
Epoch 5/10
1563/1563 [==============================] - 215s 138ms/step - loss: 1.0399 - accuracy: 0.6498
Epoch 6/10
1563/1563 [==============================] - 215s 138ms/step - loss: 0.9753 - accuracy: 0.6727
Epoch 7/10
1563/1563 [==============================] - 215s 138ms/step - loss: 0.9010 - accuracy: 0.6974
Epoch 8/10
1563/1563 [==============================] - 215s 137ms/step - loss: 0.8346 - accuracy: 0.7199
Epoch 9/10
1563/1563 [==============================] - 215s 138ms/step - loss: 0.7794 - accuracy: 0.7346
Epoch 10/10
1563/1563 [=======================

In [37]:
one_test = keras.utils.to_categorical(y_test, num_classes=10)

In [38]:
results = model.evaluate(X_test, one_test, batch_size=2)

5000/5000 [==============================] - 14s 3ms/step - loss: 1.1198 - accuracy: 0.6350
